In [ ]:
import os 
os.environ['HF_HOME'] = 'D:/huggingface_cache'
os.environ['USER_AGENT'] = 'LangChain-Search-Agent/1.0'

In [ ]:
from langchain.tools import arxiv
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper 

In [ ]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250, wiki_client=None)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

In [ ]:
api_wrapper_arxiv = ArxivAPIWrapper(
    top_k_results=1, 
    doc_content_chars_max=250,
    arxiv_search=None,  # Add this
    arxiv_exceptions=None  # Add this
)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

In [ ]:
tools = [wiki, arxiv]

In [ ]:
# Custom tools{RAG Tools}
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

In [ ]:
loader=WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250).split_documents(docs)
vectordb=FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))
reteriever=vectordb.as_retriever()
reteriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool
reteriever_tool=create_retriever_tool(reteriever,"langsmith-search", "Search any information about langsmith")
reteriever_tool.name

In [ ]:
tools = [wiki, arxiv, reteriever_tool]
tools

In [ ]:
# Run all this tools with Agent and LLM models
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")

In [ ]:
# prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
# Agents
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent

In [ ]:
# Agent executor
from langchain.agents import AgentExecutor 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})

In [ ]:
agent_executor.invoke({"input":"Tell me about attention is all you need?"})